<a href="https://colab.research.google.com/github/6balls/SoraGeneratesVideos/blob/main/aistudio_gemini_prompt_freeform_01_public.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Gemini Pro on Google AI Studio. API. Colab
Приглашаю в Телеграм общаться по это теме: https://t.me/AiExp01

Все ссылки здесь: https://t.me/AiExp01/63

## Setup

### Install & import

In [ ]:
!pip -q install google-generativeai==0.3.0
!pip -q install google-generativelanguage==0.4.0

In [ ]:
# Install the client library and import necessary modules.
#!pip install google-generativeai
import google.generativeai as genai
import json
import pathlib
import pprint
import requests
import mimetypes
from IPython.display import Markdown



### Mount Google Drive

In [ ]:
from google.colab import drive
drive.mount('/gdrive')

## Set the API key

Add your API_KEY to the secrets manager in the left pannel "🔑".

In [ ]:
from google.colab import userdata

API_KEY=userdata.get('API_KEY')

In [ ]:
# Configure the client library by providing your API key.
genai.configure(api_key=API_KEY)

### Parse the arguments

In [ ]:
import json

model = 'gemini-pro' # @param {isTemplate: true}
contents = '[{"parts": [{"text":"what\'s in this picture:"}, {"image": {"image_url": "https://storage.googleapis.com/generativeai-downloads/images/scones.jpg"}}]}]' # @param {isTemplate: true}
generation_config = "{}" # @param {isTemplate: true}
safety_settings = "{}" # @param {isTemplate: true}

contents = json.loads(contents)
generation_config = json.loads(generation_config)
safety_settings = json.loads(safety_settings)

stream = False

In [ ]:
contents

### Load image data from Drive-IDs

In [ ]:
for content in contents:
  for n, part in enumerate(content['parts']):
    if image:=part.get('image', None):
      if drive_id:=image.get('drive_id', None):
        path = next(pathlib.Path(f'/gdrive/.shortcut-targets-by-id/{drive_id}').glob('*'))
        data = path.read_bytes()
        mime_type, _ = mimetypes.guess_type(path)
      elif image_url:=image.get('image_url', None):
        response = requests.get(image_url)
        data = response.content
        mime_type = response.headers['content-type']
      else:
        raise ValueError('Either drive_id or image_url must be provided.')

      if mime_type is None:
        # Guess!
        mime_type = 'image/png'

      blob = {'data': data, 'mime_type': mime_type}
      content['parts'][n] = blob

### Call the API

In [ ]:
model = 'gemini-pro-vision'

In [ ]:
# Call the model and print the response.
gemini = genai.GenerativeModel(model_name=model)

response = gemini.generate_content(
    contents,
    generation_config=generation_config,
    safety_settings=safety_settings,
    stream=False)

In [ ]:
if generation_config.get('candidate_count', 1) == 1:
  display(Markdown(response.text))
else:
  print(response.candidates)

In [ ]:
response.candidates

In [ ]:
response.prompt_feedback

## Генерация ответана текстовый запрос

In [ ]:
model = genai.GenerativeModel('gemini-pro')
response = model.generate_content("Напишите пост в Телеграмм, чтобы сообщить, что Gemini Pro теперь доступен всем разработчикам")

In [ ]:
Markdown(response.text)